# Projet de session : Classification de feuilles

## Lucile Peltier (pell3601), Sergio Redondo (reds2401)

## IFT712 - Université de Sherbrooke

# Objectif :

Tester des méthodes de classification sur une base de données Kaggle, tout en respectant l’utilisation de validation croisée, et de recherche d’hyperparamètres afin d’identifier la meilleure solution.

Base de données : https://www.kaggle.com/c/leaf-classification/

Méthodes à tester : machines à vecteurs de support (SVM), k-plus proches voisins, Naïve Bayésienne, Arbre de décisions, Forêt Aléatoire, et Réseau de Neurones avec Perceptron. 

# Initialisation

In [1]:
# Importer outils générales
import numpy as np
import pandas as pd
import os
import time
from sklearn.metrics import precision_recall_fscore_support as metriques
from sklearn.metrics import accuracy_score as accu
from sklearn.metrics import plot_roc_curve
from tabulate import tabulate
import ipywidgets as widgets


# Importer codes spécifiques
import gestion_donnees as gd
import SVM
import k_proches_voisins
import naive_bayesienne
import arbre_decision
import foret_aleatoire
import perceptron


# Ignorer les warnings
from warnings import simplefilter
simplefilter(action='ignore')


# Lire la base de données
d_base = pd.read_csv(os.getcwd() + '/donnees/train.csv')

# Choix du modèle d'apprentissage

Choisir entre les options suivantes quel algorithme d'apprentissage utiliser.
Il ne faut pas exécuter la cellule suivante, afin que la valeur de l'algorithme choisi ne soit pas réinitialisé.

In [8]:
algorithme = widgets.Dropdown(options=['SVM', 'K-proches_voisins', \
                                       'Naive_Bayesienne','Arbre_de_decisions',\
                                       'Foret_Aleatoire','Reseau_de_neurones'],
    value='SVM',description='Algorithme :', disabled=False,)
rh = widgets.Checkbox(value=False, description='Rechercher hyperparamètres',
    disabled=False,indent=False)
display(algorithme)
display(rh)

Dropdown(description='Algorithme :', options=('SVM', 'K-proches_voisins', 'Naive_Bayesienne', 'Arbre_de_decisi…

Checkbox(value=False, description='Rechercher hyperparamètres', indent=False)

## Importer l'algorithme choisi

In [32]:
if algorithme.value == 'Reseau_de_neurones':
    classif = perceptron.Perceptron()
    
elif algorithme.value == 'SVM':
    classif = SVM.SupportVectorMachine()
    
elif algorithme.value == 'K-proches_voisins': 
    classif = k_proches_voisins.KProchesVoisins()
    
elif algorithme.value == 'Naive_Bayesienne': 
    classif = naive_bayesienne.NaiveBayes()
    
elif algorithme.value == 'Arbre_decisions': 
    classif = arbre_decision.ArbreDecision()
    
elif algorithme.value == 'Foret_Aleatoire': 
    classif = foret_aleatoire.ForetAleatoire()

## Traîtement de la base de données

In [33]:
# Séparer les données et leur cibles
g_donnees = gd.GestionDonnees(d_base)
[types, X, t] = g_donnees.lecture_donnees(d_base)
    
# Séparer les données pour test et train
x_tr, x_ts, t_tr, t_ts = g_donnees.sep_donnees(X, t)

## Entraînement

On fait l'entraînement de l'algorithme choisi. Cet entrainement dépend du choix fait pour la recherche d'hyperparamètres et du type d'algorithme, ça sera fait avec les libraries de RandomizedSearchCV ou GridSearchCV. Les valeurs d'hyperparamètres à tester sont prédefinis de manière interne.

In [34]:
debut_e = time.time() # Heure de debut pour mesurer le temps d'entraînement
classif.entrainement(x_tr, t_tr, rh.value)
fin_e = time.time() # Heure de fin pour mesurer le temps d'entraînement
print('Fin de l\'entrainement. Réalisé en %.2f secondes.'% (fin_e - debut_e),'\n')

Debut de l'entrainement perceptron avec recherche d'hyperparamètres 

Paramètres utilisés pour l'entraînement perceptron : {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 64, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.00135, 'max_fun': 15000, 'max_iter': 80, 'momentum': 0.0, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'lbfgs', 'tol': 6.000000000000001e-05, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False} 

Fin de l'entrainement. Réalisé en 117.74 secondes. 



## Prédictions et métriques de performance

On fait les prédictions pour les ensembles d'entraînement complet et de test. Ensuite, on calcule les métriques de performance d'accuracy, précision, rappel et F-score.

In [35]:
# Prédictions pour les ensembles d'entraînement et de test
predict_tr = classif.prediction(x_tr)
predict_ts = classif.prediction(x_ts)

# Métriques pour évaluer l'entraînement et test
prs_tr, rec_tr, fbeta_tr, _ = metriques(t_tr, predict_tr, average='macro')
prs_ts, rec_ts, fbeta_ts, _ = metriques(t_ts, predict_ts, average='macro')
acc_tr = accu(t_tr, predict_tr)
acc_ts = accu(t_ts, predict_ts)
tab_perform = [['Accuracy', acc_tr, acc_ts],['Précision', prs_tr, prs_ts],\
               ['Rappel', rec_tr, rec_ts],['F-Beta', fbeta_tr, fbeta_ts]]
print(tabulate(tab_perform, headers=['Métrique', 'Train', 'Test'], \
               floatfmt='.4f'))

Métrique      Train    Test
----------  -------  ------
Accuracy     1.0000  0.8548
Précision    1.0000  0.8460
Rappel       1.0000  0.8359
F-Beta       1.0000  0.8209
